In [ ]:
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
dir(tf.keras.applications.resnet)

In [ ]:
print(tf.__version__)

In [ ]:
meta = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

In [ ]:
def append_ext(fn):
    return fn+".jpg"

In [ ]:
labels = meta[['Id','Pawpularity']]

In [ ]:
labels["Id"] = labels["Id"].apply(append_ext)

In [ ]:
len(labels)/3

In [ ]:
labels.to_csv('labels.csv',encoding='utf-8', index=False)

In [ ]:
def load_train(path):
        # path /kaggle/input/petfinder-pawpularity-score/
    labels = pd.read_csv('./labels.csv')
       
    train_datagen = ImageDataGenerator(validation_split = 0.75, 
                                   rescale = 1/255.
                                   #vertical_flip = True, 
                                   #horizontal_flip = True,
                                   #rotation_range = 45,
                                   #width_shift_range=0.05,
                                   #height_shift_range=0.05
    )
    
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory= path +'train/',
        x_col='Id',
        y_col='Pawpularity',
        target_size=(224, 224),
        batch_size=4,
        class_mode='raw',
        subset='training',
        seed=42)

    return train_gen_flow

In [ ]:
path = '/kaggle/input/petfinder-pawpularity-score/'
data_gen_flow = load_train(path)

In [ ]:
# getting data from iterator after augmentation
features, target = next(data_gen_flow)

In [ ]:
model = ResNet50(weights='imagenet', 
                            # we got regression model so we dont define argument classes here
                            # the same for using pre-trained model  as we will get features and not classes
                        input_shape=(224, 224, 3),
                            # at the end of ResNet 2 layers given: GlobalAveragePooling2D & Dense. 
                            # if set include_top = False where will be no such layers
                            # set up false, because we dont need Dense layer 
                        include_top=False
                            # we initialiez weights from ResNet that has benn trained on pictures with 1000 classes
                            # if we use this initialisation results should be better than starting from randon initialization
                       )

In [ ]:
features = model.predict(data_gen_flow)

In [ ]:
features.shape

In [ ]:
features_flatten = features.reshape((features.shape[0], 7 * 7 * 2048))

In [ ]:
features_flatten[0]

In [ ]:
y = labels[['Pawpularity']]

In [ ]:
y = y[0:2478]

In [ ]:
len(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features_flatten, y, test_size=0.33, random_state=42)

In [ ]:
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train.values.ravel())

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
y_pred[0:30]

In [ ]:
regressor_poly = SVR(kernel='poly')
regressor_poly.fit(X_train,y_train.values.ravel())

In [ ]:
y_pred_poly = regressor_poly.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred_poly, squared=False)

In [ ]:
y_pred_poly

Initial run 
Changes 1: backbone is not trainable. Results:val_root_mean_squared_error: 21.1810

Changes 2: increase lr. Results:

Changes 3: TBD. Results:

Changes 4: TBD. Results:

Changes 5: TBD. Results:


